<a href="https://colab.research.google.com/github/bibleme/Aiary/blob/one_line_diary/%ED%95%9C%EC%A4%84%EC%9D%BC%EA%B8%B0_gpt_4_1_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 한 줄 일기 생성기


---


### 사용 모델
* gpt-4.1-mini
### inputs
* images
### outputs
* 사진-한줄일기 쌍 json 파일
* 한줄일기 생성 txt 파일
---
* openai api 및 drive 연결 필요


In [1]:
!pip install --upgrade openai pillow

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import base64
from openai import OpenAI
from PIL import Image

# OpenAI API 키 설정
OPENAI_API_KEY = input("Enter your API key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

client = OpenAI()

# 드라이브 기준 경로 설정
BASE_DIR = "/content/drive/MyDrive/aiary"

# 이미지가 들어있는 폴더 (예: aiary/images 폴더)
IMAGE_DIR = os.path.join(BASE_DIR, "images")

# 결과 저장 폴더 (없으면 생성)
OUTPUT_DIR = os.path.join(BASE_DIR, "outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 결과 파일 경로
PAIRS_PATH = os.path.join(OUTPUT_DIR, "one_line_pairs.jsonl")   # 이미지-문장 쌍
TEXTS_PATH = os.path.join(OUTPUT_DIR, "one_line_texts.txt")     # 문장만 모음

print("IMAGE_DIR:", IMAGE_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)


IMAGE_DIR: /content/drive/MyDrive/aiary/images
OUTPUT_DIR: /content/drive/MyDrive/aiary/outputs


In [6]:
def encode_image_to_base64(image_path: str) -> str:
    """이미지 파일을 base64 문자열로 인코딩"""
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")


def generate_one_line_diary_from_image(image_path: str) -> str:
    """
    image_path: 이미지 파일 경로
    return: 한국어 한 줄 일기 문자열
    """
    image_b64 = encode_image_to_base64(image_path)

    system_prompt = (
        "너는 부모를 위한 육아 일기 도우미야. "
        "아기 또는 아이의 사진을 보고 오늘 있었던 순간을 떠올리듯이, "
        "감성적인 한국어 한 줄 일기를 만들어주는 역할을 한다."
    )

    user_prompt = (
        "위 이미지를 보고 오늘 있었던 순간을 떠올리듯이 "
        "아이의 감정, 행동을 파악하고, 주변 사물로 상황을 파악하여 "
        "한국어로 25자 이내의 감성적인 한 줄 일기를 한 문장만 써줘. "
        "문장에 이모지를 적극적으로 활용하세요. "
    )

    response = client.chat.completions.create(
        model="gpt-4.1-mini",  # 비전 지원 미니 모델
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{image_b64}"
                        },
                    },
                ],
            },
        ],
        max_tokens=64,
        temperature=0.7,
    )

    one_line = response.choices[0].message.content.strip()
    return one_line


In [7]:
import json

# 처리할 이미지 확장자들
VALID_EXTS = {".jpg", ".jpeg", ".png", ".webp"}


def list_image_files(image_dir: str):
    image_paths = []
    for root, dirs, files in os.walk(image_dir):
        for fname in files:
            ext = os.path.splitext(fname)[1].lower()
            if ext in VALID_EXTS:
                image_paths.append(os.path.join(root, fname))
    image_paths.sort()
    return image_paths


image_paths = list_image_files(IMAGE_DIR)
print(f"총 이미지 개수: {len(image_paths)}")

pairs = []      # {"image_path": ..., "one_line": ...} 리스트
one_lines = []  # 한 줄 일기 텍스트만 모음

for idx, img_path in enumerate(image_paths, start=1):
    print(f"[{idx}/{len(image_paths)}] 처리 중: {img_path}")
    try:
        one_line = generate_one_line_diary_from_image(img_path)
        print(" → 한 줄 일기:", one_line)

        pairs.append({
            "image_path": img_path,
            "one_line": one_line,
        })
        one_lines.append(one_line)

    except Exception as e:
        print(" !! 오류 발생, 이 이미지는 건너뜀:", e)


# 이미지-문장 쌍을 JSONL로 저장
with open(PAIRS_PATH, "w", encoding="utf-8") as f:
    for item in pairs:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

# 문장만 txt로 저장 (줄 단위)
with open(TEXTS_PATH, "w", encoding="utf-8") as f:
    for line in one_lines:
        f.write(line + "\n")

print("저장 완료!")
print("쌍 파일:", PAIRS_PATH)
print("텍스트 파일:", TEXTS_PATH)


총 이미지 개수: 5
[1/5] 처리 중: /content/drive/MyDrive/aiary/images/1.jpg
 → 한 줄 일기: 따뜻한 차 안에서 과자 먹방 시작🍪🚗💕
[2/5] 처리 중: /content/drive/MyDrive/aiary/images/2.jpg
 → 한 줄 일기: 고기 냄새 가득한 따뜻한 가족 외식시간🍖🥰
[3/5] 처리 중: /content/drive/MyDrive/aiary/images/3.jpg
 → 한 줄 일기: 장난감 총과 친구, 오늘도 신나게 놀았어요🔫🤠✨
[4/5] 처리 중: /content/drive/MyDrive/aiary/images/4.jpg
 → 한 줄 일기: 따끈한 고기 냄새에 기대 가득한 밤🍖✨
[5/5] 처리 중: /content/drive/MyDrive/aiary/images/5.jpg
 → 한 줄 일기: 불꽃 속에서 아빠 손 잡고 따뜻함을 느꼈어요🔥🤗
저장 완료!
쌍 파일: /content/drive/MyDrive/aiary/outputs/one_line_pairs.jsonl
텍스트 파일: /content/drive/MyDrive/aiary/outputs/one_line_texts.txt
